In [1]:
import redis
import json
import paho.mqtt.client as mqtt
import paho.mqtt.subscribe as subscribe

In [ ]:
class RedisClient():
    def __init__(
        self,
        username : str,
        pswd : str,
        publicid : str,
        port : int,
    ):

        self.username = username
        self.pswd = pswd
        self.publicid = publicid
        self.port = port 

    def create_client(self):
        return redis.Redis(
            host=self.publicid,
            password=self.pswd,
            username=self.username,
            port=self.port
        )

    def check_for_connection(self, redis_client):
        return redis_client.ping()

    def safe_ts_create(self, redis_client, ts_name):
        try:
            redis_client.ts().create(ts_name)
        except redis.ResponseError:
            pass

    # def add_el_to_ts(redis_client, ts_name, timestamp, value):
    #     redis_client.ts().add(ts_name, timestamp, value)

In [ ]:
clientredis = RedisClient(
    username = "default",
    pswd = "53R8YAlL81zAHIEVcPjwjzcnVQoSPhzt",
    publicid = "redis-15072.c77.eu-west-1-1.ec2.cloud.redislabs.com",
    port = 15072
)
redis_client = clientredis.create_client()
clientredis.check_for_connection(redis_client=redis_client)

In [ ]:
client_mqtt = mqtt.Client()

def on_connect(client, userdata, flags, rc):
    print(f'Connected with result code {str(rc)}')
    # Subscribe to a topic when the client connects
    client.subscribe('s299266')

def on_message(client, userdata, msg):
    global redis_client, clientredis
    message = msg.payload.decode()
    message = json.loads(message)
    topic = msg.topic
    print(f"Received message from {message.get('mac_address')}")
    clientredis.safe_ts_create(redis_client=redis_client, ts_name=f"{message.get('mac_address')}:speed")
    # clientredis.safe_ts_create(redis_client=redis_client, ts_name=f"{message.get('mac_address')}:power")
    redis_client.ts().add(f"{message.get('mac_address')}:speed", int(message.get("timestamp")), message.get("speed"))
    print(redis_client.ts().get(f"{message.get('mac_address')}:speed"))
    #print(redis_client.keys())
    print("Added speed informations")
    # redis_client.ts().add(f"{message.get('mac_address')}:power", int(message.get("timestamp")), int(message.get("power_plugged")))
    # print("Added power_plugged informations")

client_mqtt.on_connect = on_connect
client_mqtt.on_message = on_message

client_mqtt.connect("mqtt.eclipseprojects.io", 1883)